In [ ]:
import sys,os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive, files
import pickle as pickle


In [ ]:
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if __name__=='__main__':
    print('Using device:', device)

Using device: cuda


In [ ]:
DATA_PATH = '/content/drive/My Drive/BiteNetProject/data_processing/'

data = pickle.load(open(os.path.join(DATA_PATH,'data.pkl'), 'rb'))


In [ ]:
## MANIPULATE AND PROCESS FEATURES
## Seqs: Get list of patient visits, where each visit is a list of medical codes 
## [patient_1[visit1...visit10]....patient_2[visit1....visit10]], each visit is a list [medcode1....medcode10]
seqs= [i[2] for i in data]

time_interval = [i[1] for i in data]



#Subtract 1 from time interval and make it sequential difference in months between visits
for patient_i, patient in enumerate(time_interval):
    time_interval[patient_i] = [(patient[visit_j] - patient[visit_j - 1])//30 for visit_j, visit in enumerate(patient) if visit_j>0]

for patient_i, patient in enumerate(time_interval):
    for visit_j, visit in enumerate(patient):
      if visit <= 1:
        patient[visit_j] = "0-1m" 
      if visit > 1 and visit <= 3:
        patient[visit_j] = "1-3m" 
      if visit > 3 and visit <= 6:
        patient[visit_j] = "1-6m" 
      if visit > 6 and visit <= 12:
        patient[visit_j] = "6-12m"
      if visit > 12:
        patient[visit_j] = "12+m"


## remove 0s from visits and patients // dont need padding for the CNN

for patient_i, patient in enumerate(seqs):
  seqs[patient_i] = [visit for visit in patient if sum(visit)>0]

for patient_i, patient in enumerate(seqs):
  for visit_j, visit in enumerate(patient):
    patient[visit_j] = [str(medcode) for medcode in visit if (medcode > 0)]
    random.shuffle(patient[visit_j])



## Append time interval to the end
for patient_i, patient in enumerate(seqs):
  for visit_j, visit in enumerate(patient):
    if visit_j <= 8:
      patient[visit_j].append(time_interval[patient_i][visit_j])



## Flatten the list of visits so each patient has a list of medcodes 

for patient_i, patient in enumerate(seqs):
  seqs[patient_i] = [medcode for visit in patient for medcode in visit][:100]
## Feature 



## Num of Codes
unique_codes = (set([code for patient in seqs for code in patient]))

num_codes = len(unique_codes)


## target label of readmission 
diag = [i[3] for i in data]



assert len(seqs) == len(diag)

In [ ]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, seqs, diag):
        
        self.x = self.convert2idx(seqs)
        self.y = diag
    
    def convert2idx(self,seqs):
        word2idx = {}
        j = 0

        for patientid, patient in enumerate(seqs):
          for visitid, visit in enumerate(patient):
            if visit in word2idx:
              patient[visitid] = word2idx[visit]
            if visit not in word2idx:
              word2idx[visit] = j
              patient[visitid] = j
              j+=1
        return seqs


    def __len__(self):
        
        return len(self.x)
    
    def __getitem__(self, index):

        return self.x[index],self.y[index]
        
data = CustomDataset(seqs, diag)


In [ ]:
from torch.utils.data.dataset import random_split

train_test_split = int(len(data)*0.8)
lengths = [train_test_split, len(data) - train_test_split]
train_data, test_data = random_split(data, lengths)


train_val_split = int(len(train_data)*0.5)
lengths = [train_val_split, len(train_data) - train_val_split]
train_data, val_data = random_split(train_data, lengths)

print(train_data)
print("Length of train dataset:", len(train_data))
print("Length of val dataset:", len(val_data))
print("Length of test dataset:", len(test_data))



Length of train dataset: 2998
Length of val dataset: 2998
Length of test dataset: 1500


In [ ]:
def collate_fn(data):
  sequences, labels = zip(*data)

  num_patients = len(sequences)

  x = torch.zeros((num_patients, 100), dtype=torch.long)
  
  y = torch.tensor(labels, dtype=torch.float)

  for ipatient, patient in enumerate(sequences):
    padding_needed = 100 - len(patient) 
    document = sequences[ipatient] + [0] * padding_needed       
    x[ipatient] = torch.tensor(document,dtype=torch.long)


  return x, y

In [ ]:
from torch.utils.data import DataLoader

def load_data(train_data, val_data, test_data, collate_fn):
    
    batch_size = 32
    ## iter will get a batch of size 32 [10 visits x 39 codes ] 

    train_loader = DataLoader(dataset = train_data, batch_size = 32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(dataset = val_data, batch_size = 32, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(dataset = test_data, batch_size = 32, shuffle=False, collate_fn=collate_fn)

    
    return train_loader, val_loader, test_loader


train_loader, val_loader, test_loader = load_data(train_data, val_data, test_data, collate_fn)






In [ ]:
class CNN(nn.Module):
    
    def __init__(self, num_codes, embedding_dim=100):
        super().__init__()

        self.embedding = nn.Embedding(num_codes, embedding_dim)
   
        self.conv1 = nn.Conv2d(in_channels = 1, kernel_size = (3,1), out_channels = 1) ## in_channels = 1, select 3 filters so 3 out_channels

        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size=(98,1))


        self.linear = nn.Linear(100,170)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        #print(x.shape) #32 100
        batch_size = x.shape[0]
        x = self.embedding(x).unsqueeze(1)
        #print("after embd",x.shape) #[32, 1, 100, 100]

        x = self.conv1(x).squeeze(1)

        #print("after conv",x.shape)

        x = self.relu(x)

        #print(x.shape)

        max_x = self.maxpool(x).squeeze(1)

        #print("after max pool",max_x.shape)

        result = self.linear(max_x)

        result = self.sigmoid(result)

        #print(result)


        return result


# load the model here
MODEL = CNN(num_codes = num_codes)
MODEL




CNN(
  (embedding): Embedding(3425, 100)
  (conv1): Conv2d(1, 1, kernel_size=(3, 1), stride=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=(98, 1), stride=(98, 1), padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=100, out_features=170, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(MODEL.parameters(), lr=0.001)



In [ ]:
from sklearn.utils.validation import indexable
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import precision_score

def eval_model(model, loader):
    model.eval()
    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, y in loader:
        y_hat = model(x)
        y_hat = y_hat > 0.5
        y_hat = y_hat.int()
        y = y.int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    precision = precision_score(y_true,y_pred,average="micro")


    return precision

In [ ]:
def train(model, train_loader, val_loader, n_epochs, print_train_results=True):
    for epoch in range(n_epochs):
      model.train()
      train_loss = 0
      for x, y in train_loader:
        loss = None
        optimizer.zero_grad()
        y_hat = model(x)

        loss = criterion(y_hat,y)
        loss.backward()
        optimizer.step()
        # your code here
        
        train_loss += loss.item()
      train_loss = train_loss / len(train_loader)
      if print_train_results==True:
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
      precision = eval_model(model, val_loader)
      if print_train_results==True:
        print('Epoch: {} \t Validation overall precision p:{:.3f}'.format(epoch+1,precision))
      

In [ ]:
n_epochs = 10
train(MODEL, train_loader, val_loader, n_epochs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


Epoch: 1 	 Training Loss: 0.297297
Epoch: 1 	 Validation overall precision p:0.593
Epoch: 2 	 Training Loss: 0.164346
Epoch: 2 	 Validation overall precision p:0.620
Epoch: 3 	 Training Loss: 0.158474
Epoch: 3 	 Validation overall precision p:0.633
Epoch: 4 	 Training Loss: 0.155890
Epoch: 4 	 Validation overall precision p:0.636
Epoch: 5 	 Training Loss: 0.154096
Epoch: 5 	 Validation overall precision p:0.651
Epoch: 6 	 Training Loss: 0.152572
Epoch: 6 	 Validation overall precision p:0.648
Epoch: 7 	 Training Loss: 0.151190
Epoch: 7 	 Validation overall precision p:0.660
Epoch: 8 	 Training Loss: 0.149960
Epoch: 8 	 Validation overall precision p:0.683
Epoch: 9 	 Training Loss: 0.148780
Epoch: 9 	 Validation overall precision p:0.672
Epoch: 10 	 Training Loss: 0.147624
Epoch: 10 	 Validation overall precision p:0.690


In [ ]:
def test(model, test_loader, test_number):
      precision = eval_model(model, test_loader)
      
      
      print('Test: test_number{} \t Test precision :{:.3f}'
              .format(test_number+1,precision))

In [ ]:
test_number = 3
for i in range(test_number):
  train_test_split = int(len(data)*0.8)
  lengths = [train_test_split, len(data) - train_test_split]
  train_data, test_data = random_split(data, lengths)


  train_val_split = int(len(train_data)*0.5)
  lengths = [train_val_split, len(train_data) - train_val_split]
  train_data, val_data = random_split(train_data, lengths)
  
  train_loader, val_loader, test_loader = load_data(train_data, val_data, test_data, collate_fn)
  
  newmodel = CNN(num_codes = num_codes)
  criterion = nn.BCELoss()
  optimizer = optim.Adam(newmodel.parameters(), lr=0.001)

  n_epochs = 10
  train(newmodel, train_loader, val_loader, n_epochs,print_train_results=False)
  test(newmodel, test_loader, i)

Test: test_number1 	 Test precision :0.690
Test: test_number2 	 Test precision :0.666
Test: test_number3 	 Test precision :0.655
